# <center><b><h1> Distribution Analysis </h1></b></center>

<p>Here we will study the distribution of the regions (148) , for each label (2) , for each feature (4).</p>
<p>For do that we will apply:</p>
<ol>
    <li>
    Anderson_darling test, wich will tell us if the distribution of a
    region is normal (parametric, Gauss-Bell) or non parametric for each label. The possible results are: 
        <ul>
            <li>
                The region x in the label A is normal as the region x in the label b,
                (label a = strong, label b = weak), if is normal in both we can make an hypothesis that
                this region for both labels has a normal distribution.
                <br><br>
                If the region has a normal distribution in the two label, we will apply a two-samples t-test
                for evaluate how much different are (the distribution of the region x in both labels). 
            </li>
        </ul>
        <ul>
            <li>
                if the region x in the label a is normal but in the label b is not normal, 
                (remember that normal is about the distribution) or vice versa, 
                <br><br>
                them we will report this result to decide the next step.
            </li>
        </ul>
        <ul>
            <li>
                otherwise if the region x in both labels has not  a normal distribution 
                (not parametric distribution) we will made a nonparametric test 
                or Mann-Whitney to know how much diferent are
                the distributions of the same region in the two labels
            </li>
        </ul>
    </li>
    <li>
         for the parametrics and not parametric regions (in both labels) we will made a graph
    </li>
</ol>

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import seaborn as sns
sns.set(color_codes=True)
from matplotlib.backends.backend_pdf import PdfPages

from scipy import stats
import numpy as np

from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr


PATH_DATASETS = "../../2_Data_preparation/1_Compute_raw_datasets/output/"
PATH_OUTPUT =  "output/"

In [2]:
def readCsvs():
    datasets = []
    constant = '_Classified'
    features = ['area', 'meancurv', 'thickness', 'volume']

    for feature in features:
        df = pd.read_csv(PATH_DATASETS + feature + constant+ '.csv', index_col=0)
        datasets.append({'name': feature, 'data': df})
    return datasets

In [3]:
def isNormal(anderson_stat, critical_values, significance_levels):
    #h0 the distribution is normal
    result = {'normal': 1, 'significance': 0}

    for critical_value, significance_level in zip(critical_values, significance_levels):
        if anderson_stat > critical_value:
            # not is normal with a significance level of: 
            result['normal'] = 0
            result['significance'] = significance_level
        else:
            break
            
    return result

# 1. Prepare the data

In [4]:
Datasets = readCsvs() #Area,MeanCurv,Thickness,Volume

In [5]:
print(Datasets[0]['data'].shape)
Datasets[1]['data'].head()

(1108, 149)


,lh_G_and_S_frontomargin_meancurv,lh_G_and_S_occipital_inf_meancurv,lh_G_and_S_paracentral_meancurv,lh_G_and_S_subcentral_meancurv,lh_G_and_S_transv_frontopol_meancurv,lh_G_and_S_cingul-Ant_meancurv,lh_G_and_S_cingul-Mid-Ant_meancurv,lh_G_and_S_cingul-Mid-Post_meancurv,lh_G_cingul-Post-dorsal_meancurv,lh_G_cingul-Post-ventral_meancurv,...,rh_S_pericallosal_meancurv,rh_S_postcentral_meancurv,rh_S_precentral-inf-part_meancurv,rh_S_precentral-sup-part_meancurv,rh_S_suborbital_meancurv,rh_S_subparietal_meancurv,rh_S_temporal_inf_meancurv,rh_S_temporal_sup_meancurv,rh_S_temporal_transverse_meancurv,class
Subject,,,,,,,,,,,,,,,,,,,,,
100206,0.177,0.165,0.144,0.178,0.198,0.160,0.170,0.171,0.219,0.217,...,0.175,0.128,0.145,0.137,0.152,0.159,0.136,0.136,0.156,1.0
100307,0.173,0.150,0.132,0.183,0.218,0.175,0.157,0.160,0.243,0.198,...,0.188,0.152,0.123,0.125,0.157,0.152,0.144,0.137,0.147,-1.0
100408,0.167,0.173,0.137,0.166,0.180,0.169,0.169,0.179,0.241,0.178,...,0.173,0.132,0.142,0.116,0.139,0.177,0.131,0.131,0.137,1.0
100610,0.138,0.153,0.116,0.139,0.157,0.133,0.151,0.154,0.196,0.163,...,0.161,0.125,0.115,0.119,0.139,0.131,0.114,0.122,0.145,1.0
101006,0.152,0.155,0.121,0.153,0.179,0.157,0.138,0.131,0.202,0.175,...,0.122,0.109,0.118,0.112,0.166,0.160,0.135,0.126,0.135,1.0


In [6]:
class_1 = 0
class_2 = 0
results_anderson_strong = []
results_anderson_weak = []

for dataset in Datasets:
    name = dataset['name']
    data = dataset['data']
    
    # copy all the subjects that belongs to the class 1 and -1 respectively
    # remember that class is a colum  in the end of the raw dataset classified
    class_1 = data[data['class'] == 1].copy()  
    class_2 = data[data['class'] == -1].copy()
    
    # the [:-1] is ommit the last character, in this case ommit the last colum that in this case is the column Class
    # you can print this and see the last part of each print to compare. 
    #print(df.columns.values)
    #print('\n')
    #print(df_stat.columns.values)
    
    df_anderson_strong = pd.DataFrame(columns=('region', 'anderson stat', 'critical values', 'significance level'))
    df_anderson_strong['region'] = data.columns.values[:-1]
    df_anderson_strong.set_index('region', inplace=True)
    
    df_anderson_weak = pd.DataFrame(columns=('region', 'anderson stat', 'critical values', 'significance level'))
    df_anderson_weak['region'] = data.columns.values[:-1]
    df_anderson_weak.set_index('region', inplace=True)
    
    for column in data.columns.values[:-1]:
        df_anderson_strong.at[column, 'anderson stat'], df_anderson_strong.at[column, 'critical values'],  df_anderson_strong.at[column, 'significance level'] = stats.anderson(class_1[column], dist = 'norm')
        
        df_anderson_weak.at[column, 'anderson stat'], df_anderson_weak.at[column, 'critical values'], df_anderson_weak.at[column, 'significance level'] = stats.anderson(class_2[column], dist = 'norm')

    results_anderson_strong.append({'name': name, 'data': df_anderson_strong})
    results_anderson_weak.append({'name': name, 'data': df_anderson_weak})

In [7]:
print("This is with the label strong")
results_anderson_strong[0]['data']

This is with the label strong


,anderson stat,critical values,significance level
region,,,
lh_G_and_S_frontomargin_area,0.472016,"[0.572, 0.651, 0.781, 0.911, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_occipital_inf_area,0.491466,"[0.572, 0.651, 0.781, 0.911, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_paracentral_area,1.89195,"[0.572, 0.651, 0.781, 0.911, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_subcentral_area,0.908611,"[0.572, 0.651, 0.781, 0.911, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_transv_frontopol_area,0.232544,"[0.572, 0.651, 0.781, 0.911, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
...,...,...,...
rh_S_suborbital_area,0.54585,"[0.572, 0.651, 0.781, 0.911, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
rh_S_subparietal_area,5.17321,"[0.572, 0.651, 0.781, 0.911, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
rh_S_temporal_inf_area,0.208243,"[0.572, 0.651, 0.781, 0.911, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"


In [8]:
print("This is with the label weak")
results_anderson_weak[0]['data']

This is with the label weak


,anderson stat,critical values,significance level
region,,,
lh_G_and_S_frontomargin_area,0.548405,"[0.572, 0.651, 0.781, 0.912, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_occipital_inf_area,1.01914,"[0.572, 0.651, 0.781, 0.912, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_paracentral_area,2.86747,"[0.572, 0.651, 0.781, 0.912, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_subcentral_area,1.92331,"[0.572, 0.651, 0.781, 0.912, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_transv_frontopol_area,1.74058,"[0.572, 0.651, 0.781, 0.912, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
...,...,...,...
rh_S_suborbital_area,0.764867,"[0.572, 0.651, 0.781, 0.912, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
rh_S_subparietal_area,4.22859,"[0.572, 0.651, 0.781, 0.912, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"
rh_S_temporal_inf_area,0.822603,"[0.572, 0.651, 0.781, 0.912, 1.084]","[15.0, 10.0, 5.0, 2.5, 1.0]"


In [9]:
results_anderson = []
for strong_result, weak_result in zip(results_anderson_strong, results_anderson_weak):
    
    name = strong_result['name']
    
    result_anderson = pd.DataFrame(columns=('region', 'normal in strong', 'normal in weak', 'normal in both'))
    result_anderson['region'] = strong_result['data'].index.values
    result_anderson.set_index('region', inplace=True)
    
    for column in strong_result['data'].index.values:
        Anderson_stat_strong = strong_result['data'].at[column, 'anderson stat']
        Anderson_stat_weak = weak_result['data'].at[column, 'anderson stat']
        
        critical_values_strong = strong_result['data'].at[column, 'critical values']
        critical_values_weak = weak_result['data'].at[column, 'critical values']
        
        significance_values = strong_result['data'].at[column, 'significance level']
        
        
        normal_strong = isNormal(Anderson_stat_strong, critical_values_strong, significance_values)
        if normal_strong['normal']:
            result_anderson.at[column, 'normal in strong'] = 1
        else:
            result_anderson.at[column, 'normal in strong'] = [0,normal_strong['significance']]
            
        normal_weak = isNormal(Anderson_stat_weak, critical_values_weak, significance_values)
        
        if normal_weak['normal']:
            result_anderson.at[column, 'normal in weak'] = 1
        else:
            result_anderson.at[column, 'normal in weak'] = [0, normal_weak['significance']]
            
            
        if normal_weak['normal'] and normal_strong['normal']:
            result_anderson.at[column, 'normal in both'] = 1
        else:
            result_anderson.at[column, 'normal in both'] = 0
            
    results_anderson.append(result_anderson)

<h3>Anderson result test for nomality</h3>
<p>
  whit this test we have classified each distribution, remember that each region has 2 distributions, one for each label.
</p>
<p>
    1 = means normal, actually, the H0 of this test is that the distribution is normal. <br>
    0 = means not normal, and the other value is the significance level. 
</p>

In [10]:
results_anderson[3]

,normal in strong,normal in weak,normal in both
region,,,
lh_G_and_S_frontomargin_volume,1,1,1
lh_G_and_S_occipital_inf_volume,"[0, 5.0]","[0, 1.0]",0
lh_G_and_S_paracentral_volume,"[0, 1.0]","[0, 1.0]",0
lh_G_and_S_subcentral_volume,"[0, 1.0]","[0, 1.0]",0
lh_G_and_S_transv_frontopol_volume,"[0, 1.0]","[0, 1.0]",0
...,...,...,...
rh_S_suborbital_volume,"[0, 1.0]","[0, 1.0]",0
rh_S_subparietal_volume,"[0, 1.0]","[0, 1.0]",0
rh_S_temporal_inf_volume,1,"[0, 2.5]",0


In [11]:
class_1 = 0
class_2 = 0
statistical_results = []
ocurrences = []
for dataset, anderson_result in zip(Datasets, results_anderson):
    name = dataset['name']
    data = dataset['data']
    
    # copy all the subjects that belongs to the class 1 and -1 respectively
    # remember that class is a colum  in the end of the raw dataset classified
    class_1 = data[data['class'] == 1].copy()  
    class_2 = data[data['class'] == -1].copy()
    
    # the [:-1] is ommit the last character, in this case ommit the last colum that in this case is the column Class
    # you can print this and see the last part of each print to compare. 
    #print(df.columns.values)
    #print('\n')
    #print(df_stat.columns.values)
    
    df_stat = pd.DataFrame(columns=data.columns.values[:-1])
    
    df_stat.at['t(Xi)', :] = 0
    df_stat.at['MW STAT', :] = 0
    df_stat.at['p-value', :] = 0
    df_stat.at['Dfferent distribution', :] = 0
    #describing distribution
    df_stat.at['mean_strong', :] = 0
    df_stat.at['median_strong', :] = 0
    df_stat.at['SD_strong', :] = 0
    
    df_stat.at['mean_weak', :] = 0
    df_stat.at['median_weak', :] = 0
    df_stat.at['SD_weak', :] = 0
    
    normal = 0
    no_normal = 0
    mix = 0

    #df_stat.head()

    for column in data.columns.values[:-1]:
        normal_weak = anderson_result.at[column, 'normal in weak']
        normal_strong = anderson_result.at[column, 'normal in strong']
        
        df_stat.at['mean_strong', column] = class_1[column].mean()
        df_stat.at['median_strong', column] = class_1[column].median()
        df_stat.at['SD_strong', column] = np.std(class_1[column])
        
        df_stat.at['mean_weak', column] = class_2[column].mean()
        df_stat.at['median_weak', column] = class_2[column].median()
        df_stat.at['SD_weak', column] = np.std(class_2[column])
        

        if  isinstance(normal_weak, list) :
            normal_weak = normal_weak[0]
            
        if  isinstance(normal_strong, list):
            normal_strong = normal_strong[0]
            
        if normal_weak and normal_strong:
            df_stat.at['t(Xi)', column], df_stat.at['p-value', column] = ttest_ind(class_1[column], class_2[column], equal_var=True)
            normal = normal + 1
        elif not normal_weak and not normal_strong:
            df_stat.at['MW STAT', column], df_stat.at['p-value', column] = stats.mannwhitneyu(class_1[column], class_2[column])
            no_normal = no_normal + 1
        else:
            df_stat.at['Dfferent distribution', column] = 1
            df_stat.at['MW STAT', column], df_stat.at['p-value', column] = stats.mannwhitneyu(class_1[column], class_2[column])            
            mix = mix + 1
    
    # T = Transpose index and columns 
    df_stat = df_stat.T.sort_values(by=['p-value'], ascending = True)
    ocurrence = [normal, no_normal, mix] 
    ocurrences.append({'name': name, 'data': ocurrence})
    # returns the las n rows, by defect n = 5 
    #df_stat.tail()
    statistical_results.append(df_stat)
    #saving the results 
    df_stat.to_csv(PATH_OUTPUT + "Estatistical-test_" + name + ".csv")

In [18]:
print("name ------> normal - no normal - mixture")
print()
for ocurrence in ocurrences:
    name = ocurrence['name']
    data = ocurrence['data']
    
    print(name + " ----> " + str(data))

name ------> normal - no normal - mixture

area ----> [17, 84, 47]
meancurv ----> [9, 113, 26]
thickness ----> [42, 60, 46]
volume ----> [10, 98, 40]


<h3> Understanding the result</h3>
<p> 
   with this table we could apreciate the t-test results for the regions in wich, the region has a normal distribution in the 2 labels, and if the region has not a normal distributon we could see the Mean Whitney U test, is the non parametric version of the T-test, also we coul see a flag that tell us if the region has a different distribution between the two labels. 
</p>
<p>
    For mean withney U tes
    306900 the critical value of u less than this value is sifnificant.<br>
    0 = mean equal distribution<br>
    close to the critical value  is totally different<br>
   
</p>

In [15]:
pd.set_option('display.max_columns',144, 'display.max_rows',144)

In [16]:
statistical_results[0].head(144)

,t(Xi),MW STAT,p-value,Dfferent distribution,mean_strong,median_strong,SD_strong,mean_weak,median_weak,SD_weak
lh_S_circular_insula_sup_area,13.563618,0.0,7.019255e-39,0.0,1270.649091,1267.0,140.901573,1163.489247,1165.5,121.241764
rh_S_temporal_sup_area,0.000000,86192.5,7.318770e-37,1.0,4486.590909,4468.0,577.041818,4058.951613,4041.5,453.584785
lh_G_front_sup_area,0.000000,86911.0,4.027674e-36,1.0,5147.585455,5141.5,634.875744,4675.109319,4646.0,531.526614
lh_S_temporal_sup_area,0.000000,87196.0,7.881951e-36,1.0,4059.396364,4065.5,536.005433,3664.353047,3652.0,430.791073
rh_G_and_S_cingul-Ant_area,0.000000,88830.5,3.507518e-34,0.0,2067.025455,2083.5,270.456912,1870.354839,1865.0,240.161728
rh_G_front_sup_area,0.000000,89619.5,2.119633e-33,1.0,4895.070909,4888.0,581.927290,4475.756272,4447.0,522.377541
lh_G_temporal_middle_area,0.000000,89893.5,3.937948e-33,1.0,2079.336364,2079.5,306.723716,1861.550179,1842.0,264.118674
lh_G_insular_short_area,0.000000,90230.0,8.386557e-33,1.0,480.903636,476.0,78.187210,425.077061,414.0,70.110252
rh_S_circular_insula_sup_area,0.000000,90330.0,1.050207e-32,1.0,998.721818,999.0,125.609919,911.643369,906.0,107.537507
lh_G_pariet_inf-Angular_area,0.000000,91008.5,4.769005e-32,0.0,1779.423636,1759.0,298.406289,1572.311828,1558.5,235.890769
